In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_dir = '/mnt/d/machinelearning/plant_image_classifiaction/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train' #add train dataset path
valid_dir = '/mnt/d/machinelearning/plant_image_classifiaction/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid' #add validation dataset path
test_dir = '/mnt/d/machinelearning/plant_image_classifiaction/test/test' #add test dataset path

In [ ]:
from collections import Counter
def plot(generator, title):
    class_indices = generator.class_indices
    labels = list(class_indices.keys())
    counts = dict(Counter(generator.classes))

    plt.figure(figsize=(10,5))
    plt.bar(range(len(counts)), list(counts.values()), tick_label = labels)
    plt.title(title)
    plt.xlabel('classes')
    plt.ylabel('no. of images')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
import tensorflow as tf

# Use image_dataset_from_directory without class_mode
train_gen = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',   # Automatically infers the class labels
    batch_size=32,
    image_size=(128, 128),  # Adjust image size as needed
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,  # If you want to use validation split, set subset to 'training' or 'validation'
    interpolation='bilinear',
    follow_links=False
)

# Example for validation generator (if needed)
valid_gen = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    labels='inferred',   # Automatically infers the class labels
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False
)

test_gen = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    labels='inferred',   # Automatically infers the class labels
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False
)


In [ ]:
# Convert class names to a dictionary where index is the key, and class name is the value
class_dict = {i: class_name for i, class_name in enumerate(train_gen.class_names)}

print(class_dict)


In [ ]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, Multiply, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
import tensorflow as tf

class SpatialAttention(Layer):
    def __init__(self, kernel_size=7, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.kernel_size = kernel_size
        self.conv = Conv2D(1, kernel_size=self.kernel_size, strides=1, padding='same', activation='sigmoid')

    def call(self, input_feature):
        avg_pool = tf.reduce_mean(input_feature, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(input_feature, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        attention = self.conv(concat)
        return Multiply()([input_feature, attention])

    def get_config(self):
        config = super(SpatialAttention, self).get_config()
        config.update({
            'kernel_size': self.kernel_size
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


# Load the VGG16 base model without global pooling
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze the base model

# Apply Spatial Attention after the base model's output
x = base_model.output
x = SpatialAttention()(x)  # Apply custom spatial attention to the feature map

# Global Average Pooling after attention
x = GlobalAveragePooling2D()(x)

# Flatten and add Dense Layers
x = Dense(512, activation='relu')(x)  # Add a fully connected layer
x = Dropout(0.5)(x)  # Add Dropout to prevent overfitting

# Final output layer
# num_classes = len(train_gen.class_indices)  # Number of output classes
output = Dense(38, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='efficientnetb3.png')

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

# Define the callbacks
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor
    factor=0.5,          # Factor by which the learning rate will be reduced
    patience=5,          # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6,         # Lower bound on the learning rate
    verbose=1            # Verbosity mode
)

early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,         # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # Verbosity mode
    restore_best_weights=True  # Restore the model weights from the epoch with the best value of the monitored quantity
)

# Fit the model with the callbacks
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    # steps_per_epoch=len(train_gen),
    # validation_steps=len(valid_gen),
    callbacks=[reduce_lr, early_stopping]  # Add the callbacks here
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Plotting training & validation accuracy and loss
def plot_training_history(history):
    # Plot accuracy
    plt.figure(figsize=(14, 5))
    
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Call this function after training
plot_training_history(history)

In [ ]:
predictions = model.predict(test_gen)
predicted_classes = np.argmax(predictions, axis=1)  # Convert from one-hot encoded output to class index

# Get true labels from the test generator
true_classes = np.concatenate([y for x, y in test_gen], axis=0)

# Check if true_classes are one-hot encoded or integer encoded
if true_classes.ndim > 1:
    # If the labels are one-hot encoded, convert them to class index
    true_classes = np.argmax(true_classes, axis=1)

# Create the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_gen.class_names, yticklabels=test_gen.class_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Generate a classification report
class_report = classification_report(true_classes, predicted_classes, target_names=test_gen.class_names)
print('Classification Report:\n', class_report)

In [ ]:
model.save('EfficeintNetB3.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Ensure you register the custom layer when loading
model = load_model('EfficeintNetB3.keras', custom_objects={'SpatialAttention': SpatialAttention})


In [ ]:
import cv2
image_path = './test/test/Apple__Apple_scab/AppleScab1.JPG'
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.title('Test Image')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(128,128))
input_array = tf.keras.preprocessing.image.img_to_array(image)
input_array = np.array([input_array])

In [ ]:
prediction = model.predict(input_array)
prediction

In [ ]:
results = np.argmax(prediction)

In [ ]:
import os
test_dir = '/mnt/d/machinelearning/plant_image_classifiaction/test/test'
class_name = os.listdir(test_dir)

In [ ]:
model_prediction = class_name[results]
print(model_prediction)